In [2]:
import numpy as np
import pandas as pd

In [3]:
s1011 = pd.read_csv(
    "data/schema1011.csv",
    skiprows = 1,
    quotechar = "\"",
    na_values = "NA"
)
s1011["season"] = "s1011"

In [4]:
s1011.head(3)

,matchnr_link,matchnr_link/_text,matchnr_link/_title,match_link_1,match_link_1/_text,match_link_1/_title,match_link_2,match_link_2/_text,match_link_2/_title,resultat_link,...,resultat_link/_title,rapport_link,rapport_link/_text,rapport_link/_title,tid_value,tid_value_dates,match_label,publik_number,domare_value,season
0,http://www.elitrapport.se/Data/report99.html,20000001102,NaN,http://www.elitrapport.se/team.asp?teamID=115,Villa Lidköping BK,NaN,http://www.elitrapport.se/team.asp?teamID=145,Bollnäs GIF,NaN,http://www.elitrapport.se/Data/report99.html,...,NaN,http://www.elitrapport.se/Data/report99.html,Matchrapport,NaN,2010-10-27 19:00,1288137600000,-,3021,Roland Fager,s1011
1,http://www.elitrapport.se/Data/report106.html,20000001111,NaN,http://www.elitrapport.se/team.asp?teamID=116,IK Sirius Bandyklubb,NaN,http://www.elitrapport.se/team.asp?teamID=137,Sandvikens AIK,NaN,http://www.elitrapport.se/Data/report106.html,...,NaN,http://www.elitrapport.se/Data/report106.html,Matchrapport,NaN,2010-10-27 19:00,1288137600000,-,1759,Thomas Nordin,s1011
2,http://www.elitrapport.se/Data/report22.html,20000001027,NaN,http://www.elitrapport.se/team.asp?teamID=132,Västerås SK/BK,NaN,http://www.elitrapport.se/team.asp?teamID=210,IF Boltic,NaN,http://www.elitrapport.se/Data/report22.html,...,NaN,http://www.elitrapport.se/Data/report22.html,Matchrapport,NaN,2010-10-28 19:00,1288224000000,-,1836,Ronny Andersson,s1011


In [5]:
s1112 = pd.read_csv(
    "data/schema1112.csv",
    skiprows = 1,
    quotechar = "\"",
    na_values = "NA"
)
s1112["season"] = "s1112"

s1213 = pd.read_csv(
    "data/schema1213.csv",
    skiprows = 1,
    quotechar = "\"",
    na_values = "NA"
)
s1213["season"] = "s1213"

s1314 = pd.read_csv(
    "data/schema1314.csv",
    skiprows = 1,
    quotechar = "\"",
    na_values = "NA"
)
s1314["season"] = "s1314"

s1415 = pd.read_csv(
    "data/schema1415.csv",
    skiprows = 1,
    quotechar = "\"",
    na_values = "NA"
)
s1415["season"] = "s1415"

In [6]:
# sanity check of same columns
assert all(s1011.columns == s1112.columns)
assert all(s1112.columns == s1213.columns)
assert all(s1213.columns == s1314.columns)
assert all(s1213.columns == s1415.columns)

In [7]:
# remove crap
games = s1011.append(s1112.append(s1213.append(s1314.append(s1415)))).drop(
    ["matchnr_link", "matchnr_link/_text","matchnr_link/_title", "match_link_1","match_link_1/_title",
     "match_link_2", "match_link_2/_title", "resultat_link", "resultat_link/_title", "rapport_link",
     "rapport_link/_text", "rapport_link/_title", "tid_value_dates", "match_label"],
    1
)

# pretty column names
games.rename(columns=
             {"match_link_1/_text": "home",
              "match_link_2/_text": "away",
              "resultat_link/_text": "score",
              "publik_number": "attend",
              "domare_value": "ref"
             }, inplace=True)

games.head(3)

,home,away,score,tid_value,attend,ref,season
0,Villa Lidköping BK,Bollnäs GIF,8 - 6,2010-10-27 19:00,3021,Roland Fager,s1011
1,IK Sirius Bandyklubb,Sandvikens AIK,5 - 1,2010-10-27 19:00,1759,Thomas Nordin,s1011
2,Västerås SK/BK,IF Boltic,9 - 3,2010-10-28 19:00,1836,Ronny Andersson,s1011


In [8]:
# missing any rows?
assert len(s1011) + len(s1112) + len(s1213) + len(s1314) + len(s1415) == len(games)

In [9]:
# convert to datetime
from datetime import datetime
to_datetime = lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S')

games["start_datetime"] = [x + ":00" for x in games.tid_value]
games["start_datetime"] = games.start_datetime.apply(to_datetime)
games["start_date"] = [x.split(" ")[0] for x in games.tid_value]
games["start_time"] = [x.split(" ")[1]+":00" for x in games.tid_value]
games.drop("tid_value", 1, inplace=True)
games.head()

,home,away,score,attend,ref,season,start_datetime,start_date,start_time
0,Villa Lidköping BK,Bollnäs GIF,8 - 6,3021,Roland Fager,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00
1,IK Sirius Bandyklubb,Sandvikens AIK,5 - 1,1759,Thomas Nordin,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00
2,Västerås SK/BK,IF Boltic,9 - 3,1836,Ronny Andersson,s1011,2010-10-28 19:00:00,2010-10-28,19:00:00
3,Hammarby IF,Edsbyns IF Bandy,6 - 5,3511,Håkan Sjösten,s1011,2010-10-29 19:00:00,2010-10-29,19:00:00
4,IFK Kungälv,Broberg/Söderhamn,2 - 7,821,Mikael Karlsson (Grästorp),s1011,2010-10-29 19:00:00,2010-10-29,19:00:00


In [10]:
# cleanup score
games["home_score"] = [score.split(" - ")[0] for score in games.score.values.tolist()]
games["away_score"] = [score.split(" - ")[1] for score in games.score.values.tolist()]
games[["home_score", "away_score"]] = games[["home_score", "away_score"]].astype(int)
games.head()

,home,away,score,attend,ref,season,start_datetime,start_date,start_time,home_score,away_score
0,Villa Lidköping BK,Bollnäs GIF,8 - 6,3021,Roland Fager,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00,8,6
1,IK Sirius Bandyklubb,Sandvikens AIK,5 - 1,1759,Thomas Nordin,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00,5,1
2,Västerås SK/BK,IF Boltic,9 - 3,1836,Ronny Andersson,s1011,2010-10-28 19:00:00,2010-10-28,19:00:00,9,3
3,Hammarby IF,Edsbyns IF Bandy,6 - 5,3511,Håkan Sjösten,s1011,2010-10-29 19:00:00,2010-10-29,19:00:00,6,5
4,IFK Kungälv,Broberg/Söderhamn,2 - 7,821,Mikael Karlsson (Grästorp),s1011,2010-10-29 19:00:00,2010-10-29,19:00:00,2,7


In [11]:
# clean up names
correct_teams = {
    # erroneous : correct
    "Kalix BF": "Kalix Bandy",
    "IK Sirius Bandyklubb": "IK Sirius",
    "Edsbyns IF Bandy": "Edsbyns IF",
    "Sandvikens AIK": "Sandvikens AIK/BK",
    "Tillberga Bandy": "TB Västerås"
}

for err, cor in correct_teams.iteritems():
    games.loc[games.home == err, "home"] = cor
    games.loc[games.away == err, "away"] = cor

In [12]:
teams = 0
teams = games.home.unique()
teams = pd.DataFrame(teams, columns=["team"])
teams["team_tk"] = teams.index + 1
teams.head(50)

,team,team_tk
0,Villa Lidköping BK,1
1,IK Sirius,2
2,Västerås SK/BK,3
3,Hammarby IF,4
4,IFK Kungälv,5
5,Bollnäs GIF,6
6,IFK Vänersborg,7
7,Edsbyns IF,8
8,TB Västerås,9
9,Vetlanda BK,10


In [13]:
games = pd.merge(games, teams, left_on="home", right_on="team", how="left").drop("team",1).rename(
    columns={"team_tk": "home_tk"})
games = pd.merge(games, teams, left_on="away", right_on="team", how="left").drop("team",1).rename(
    columns={"team_tk": "away_tk"})
games.head()

,home,away,score,attend,ref,season,start_datetime,start_date,start_time,home_score,away_score,home_tk,away_tk
0,Villa Lidköping BK,Bollnäs GIF,8 - 6,3021,Roland Fager,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00,8,6,1,6
1,IK Sirius,Sandvikens AIK/BK,5 - 1,1759,Thomas Nordin,s1011,2010-10-27 19:00:00,2010-10-27,19:00:00,5,1,2,13
2,Västerås SK/BK,IF Boltic,9 - 3,1836,Ronny Andersson,s1011,2010-10-28 19:00:00,2010-10-28,19:00:00,9,3,3,11
3,Hammarby IF,Edsbyns IF,6 - 5,3511,Håkan Sjösten,s1011,2010-10-29 19:00:00,2010-10-29,19:00:00,6,5,4,8
4,IFK Kungälv,Broberg/Söderhamn,2 - 7,821,Mikael Karlsson (Grästorp),s1011,2010-10-29 19:00:00,2010-10-29,19:00:00,2,7,5,12


In [29]:
def outcome(score):
    home_score = score.split(" - ")[0]
    away_score = score.split(" - ")[1]
    if home_score > away_score:
        outcome = '1'
    elif home_score < away_score:
        outcome = '2'
    else:
        outcome = 'x'
    return outcome

games["outcome"] = games.score.apply(outcome)

In [30]:
games.to_csv("data/games-clean.csv", index=False, quoting=1, quotechar="\"")